In [0]:
#!unzip train_posters.zip
#!unzip test_posters.zip

Archive:  train_posters.zip
replace train_posters/100114.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  test_posters.zip
replace test_posters/100388.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
import os
import cv2
import matplotlib.pyplot as plt

# Read csv data
# Reorder the labels to match the order of the images

csv_data = pd.read_csv('train_data.csv').as_matrix()

csv_data = csv_data[csv_data[:,1].argsort()] # csv_data[:,1].argsort() returns indices that sort movies by imdb # 
genres = csv_data[:,-1]


test_data_csv = pd.read_csv('test_data.csv').as_matrix()

# One-hot encode genres column

train_labels = to_categorical(np.array(genres))
#print("Label for first training example: {}".format(genres[0]))
#print("One-hot encoded label for first training example: {}".format(train_labels[0]))

In [0]:
train_data = 'train_posters'
test_data = 'test_posters'

def preprocess_training_data():
    train_images = []
    image_num = 0

    for ind,i in enumerate(csv_data[:,1]):

        path = os.path.join(train_data,str(i) + ".jpg")
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (64,64))

        train_images.append(np.array(img)/255)
    return train_images


def preprocess_test_data():
    test_images = []
    for ind,i in enumerate(test_data_csv[:,1]):

        path = os.path.join(test_data,str(i) + ".jpg")
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (64,64))
        test_images.append(np.array(img)/255)
            
    return test_images
    
preprocessed_train = preprocess_training_data()
preprocessed_test = preprocess_test_data()


x_train = np.array(preprocessed_train).reshape(-1,64,64,1)
y_train = train_labels

x_test = np.array(preprocessed_test).reshape(-1,64,64,1)

In [0]:
# Display training example #1371 Death Note, a well-acclaimed Hollywood adaption of the Death Note anime (jk)
# Display movie poster and associated label and title

# Feel free to change the train_ind and see how the preprocessing affect the images
train_ind = 1371
plt.imshow(preprocessed_train[train_ind])
print(csv_data[:,1][train_ind])
print(y_train[train_ind])
print(csv_data[:,(3,4,5)])

In [0]:
# Displaying test example #200 
# Remember there is no genre label or title associated with this image
# We are trying to predict the labels! 

plt.imshow(preprocessed_test[200])
print("Test example #200")

In [0]:
# CNN

from keras import Sequential
from keras.layers import InputLayer, Conv2D, MaxPool2D, Flatten, Dense


model = Sequential()

model.add(InputLayer(input_shape=[64,64,1]))
model.add(Conv2D(filters=32,kernel_size=(3,3),padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2),padding='same'))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


model.fit(x=x_train, y=y_train, epochs=8, batch_size=50)
model.summary()

In [0]:
# cnn prediction (dec values)

model.save("movie_classifier.h5py")
cnn_predict = model.predict(x_test, batch_size=32, verbose=0, steps=None)
cnn_predict

In [0]:
# Using cross validation to find the best k for KNN

from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score
import numpy as np

genre_list = genres.tolist()

# creating odd list of K for KNN
klist = list(range(1,50))

# subsetting just the odd ones
neighbors = filter(lambda x: x % 2 != 0, klist)

# empty list that will hold cv scores
cv_scores = []

# perform 10-fold cross validation
for k in neighbors:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, csv_data[:,(4,5)], genre_list, cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())
#print(cv_scores)
#print("cv_scores mean:{}".format(np.mean(cv_scores)))
print("k best =",cv_scores.index(max(cv_scores)))
print("accuracy =",cv_scores[23])

NameError: ignored

In [0]:
# KNN

from sklearn.neighbors import KNeighborsClassifier

genre_list = genres.tolist()

# KNN for runtime and score

neigh = KNeighborsClassifier(n_neighbors=23)
neigh.fit(csv_data[:,(4,5)], genre_list)
neigh_predict = neigh.predict(test_data_csv[:,(4,5)])  # neigh_predict is the KNN prediction for test data

NameError: ignored

In [0]:
from sklearn.svm import SVC 
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score
import numpy as np

genre_list = genres.tolist()

#linear
#svclassifier = SVC(kernel='linear',C=4) 

#polynomial 
#svclassifier = SVC(kernel='poly', degree=3)

#gaussian meh
#svclassifier = SVC(kernel='rbf', C=1, gamma=0.1) 

#sigmoid not good
#svclassifier = SVC(kernel='sigmoid')

#svm_scores = cross_val_score(svclassifier, csv_data[:,(3,4,5)], genre_list, cv=10, scoring='accuracy')
#print(svm_scores)

svclassifier.fit(csv_data[:,(3,4,5)], genre_list) 
y_svm = svclassifier.predict(test_data_csv[:,(3,4,5)]) 


In [ ]:
# Final Predictions
# Decide not to use Naive Bayes because it made the accuracy worse
# COMBINE CNN, KNN, and SVM

one_hot1 = to_categorical(neigh_predict)*0.4 # one-hot encode neigh_predict and multiply by 0.4 (maybe a diff value will work better, idk)
one_hot2 = to_categorical(y_svm)*0.7
predict = one_hot1 + one_hot2 + cnn_predict # Add one_hot and predict (output from the conv neural network)

In [0]:
# write test predictions in csv file

from google.colab import files
import numpy as np
import csv
predict_class = np.argmax(predict,axis=1)
with open('out.csv','w') as f:
    writer = csv.writer(f)
    for val in predict_class:
        writer.writerows(str(val))    
f.close()

files.download("out.csv") # download csv file of test predictions